In [1]:
import pandas as pd
import numpy as np

import math

In [2]:
s1314 = pd.read_csv(r'C:\Users\George\Desktop\football-match-prediction-using-ml-techniques\data\Bundesliga\D1-1314.csv')
s1415 = pd.read_csv(r'C:\Users\George\Desktop\football-match-prediction-using-ml-techniques\data\Bundesliga\bundesliga-1415.csv')
s1516 = pd.read_csv(r'C:\Users\George\Desktop\football-match-prediction-using-ml-techniques\data\Bundesliga\bundesliga-1516.csv')
s1617 = pd.read_csv(r'C:\Users\George\Desktop\football-match-prediction-using-ml-techniques\data\Bundesliga\bundesliga-1617.csv')
s1718 = pd.read_csv(r'C:\Users\George\Desktop\football-match-prediction-using-ml-techniques\data\Bundesliga\bundesliga-1718.csv')
s1819 = pd.read_csv(r'C:\Users\George\Desktop\football-match-prediction-using-ml-techniques\data\Bundesliga\bundesliga-1819.csv')
s1920 = pd.read_csv(r'C:\Users\George\Desktop\football-match-prediction-using-ml-techniques\data\Bundesliga\bundesliga-1920.csv')

In [3]:
#Winner of last season game between the two team at the certain ground
def ls_winner(dfl,df):
    
    df = pd.merge(dfl,df, how='right', left_on=['HomeTeam','AwayTeam'], right_on=['HomeTeam','AwayTeam'])
    
    df['result_ls'] = 'NG' 
    df.loc[df['FTR_x']=='H','result_ls'] = 'HomeT'
    df.loc[df['FTR_x']=='D','result_ls'] = 'Draw'
    df.loc[df['FTR_x']=='A','result_ls'] = 'AwayT'

    
    return df['result_ls']

In [4]:
s1415['result_ls'] = ls_winner(s1314,s1415)
s1516['result_ls'] = ls_winner(s1415,s1516)
s1617['result_ls'] = ls_winner(s1516,s1617)
s1718['result_ls'] = ls_winner(s1617,s1718)
s1819['result_ls'] = ls_winner(s1718,s1819)
s1920['result_ls'] = ls_winner(s1819,s1920)

In [5]:
#Average shots that a team makes from fixture to fixture
def avg_team_shots_on_target(df):
    teams = {key:[0] for key in df['HomeTeam'].unique()}
    
    j=0
    
    for i,row in df.iterrows():
       
        if j==0:
            teams[row['HomeTeam']].append(round((teams[row['HomeTeam']][-1] + row['HST']),2))
            teams[row['AwayTeam']].append(round((teams[row['AwayTeam']][-1] + row['AST']),2))
        else:
            teams[row['HomeTeam']].append(round((teams[row['HomeTeam']][-1] + row['HST'])/j,2))
            teams[row['AwayTeam']].append(round((teams[row['AwayTeam']][-1] + row['AST'])/j,2))
            
        #there are 9 fixtures at bundsliga
        if (i % 8)== 0:
            j = j + 1
        
    df['avgHST'] = 0
    df['avgAST'] = 0
    
    for i, row in df.iterrows():
    
        df.at[i, 'avgHST'] = teams[row['HomeTeam']].pop(0)
        df.at[i, 'avgAST'] = teams[row['AwayTeam']].pop(0)
  
    return df

In [6]:
s1415 = avg_team_shots_on_target(s1415)
s1516 = avg_team_shots_on_target(s1516)
s1617 = avg_team_shots_on_target(s1617)
s1718 = avg_team_shots_on_target(s1718)
s1819 = avg_team_shots_on_target(s1819)
s1920 = avg_team_shots_on_target(s1920)

In [7]:
#rolling average of shots on target a team makes in a time window of 3 games.
def roll_avg_ST(df):
    
    #Dataframe of home team matches
    home_df = df.groupby(['Date','HomeTeam']).sum()['HST'].reset_index()
    home_df.columns = ['Date','Team','ST']
    
    #Dataframe of away team matches
    away_df = df.groupby(['Date','AwayTeam']).sum()['AST'].reset_index()
    away_df.columns = ['Date','Team','ST']
    
    #Concatenation of the two dataframes and then we sort it by date ascending
    cdf = pd.concat([home_df,away_df],ignore_index=True)
    cdf = cdf.sort_values('Date',ascending=True)

    #calculation of rolling avg for each team
    cdf['ravg_ST'] = cdf.groupby('Team')['ST'].apply(lambda x: x.rolling(window=3,closed='left').mean())
    
    #creation of the two columns
    df = pd.merge(df,cdf,how='inner',left_on=['Date','HomeTeam'],right_on=['Date','Team'])
    df.rename(columns={'ravg_ST':'l3_ravg_HTST'},inplace=True)
    df = df.drop(['Team','ST'],axis=1)
    
    df = pd.merge(df,cdf,how='inner',left_on=['Date','AwayTeam'],right_on=['Date','Team'])
    df.rename(columns={'ravg_ST':'l3_ravg_ATST'},inplace=True)
    df = df.drop(['Team','ST'],axis=1)


    return df

In [8]:
s1415 = roll_avg_ST(s1415)
s1516 = roll_avg_ST(s1516)
s1617 = roll_avg_ST(s1617)
s1718 = roll_avg_ST(s1718)
s1819 = roll_avg_ST(s1819)
s1920 = roll_avg_ST(s1920)

In [9]:
#rolling average of shots on target a team makes in a time window of 3 games. Only for home or away matches.
def avg_shots_on_target_h_or_a(df): 
        
    df['avgl3_HST_ho'] = df.groupby('HomeTeam')["HST"].apply(lambda x: x.rolling(window=3,closed='left').mean()).fillna(0) 
    df['avgl3_AST_ao'] = df.groupby('AwayTeam')['AST'].apply(lambda x: x.rolling(window=3,closed='left').mean()).fillna(0)
    
    
    return df

In [10]:
s1415 = avg_shots_on_target_h_or_a(s1415)
s1516 = avg_shots_on_target_h_or_a(s1516)
s1617 = avg_shots_on_target_h_or_a(s1617)
s1718 = avg_shots_on_target_h_or_a(s1718)
s1819 = avg_shots_on_target_h_or_a(s1819)
s1920 = avg_shots_on_target_h_or_a(s1920)

In [11]:
#Average corner a team earns from fixture to fixture 
def avg_team_corner(df):
    teams = {key:[0] for key in df['HomeTeam'].unique()}
    
    j=0
    
    for i,row in df.iterrows():
       
        if j==0:
            teams[row['HomeTeam']].append(round((teams[row['HomeTeam']][-1] + row['HC']),2))
            teams[row['AwayTeam']].append(round((teams[row['AwayTeam']][-1] + row['AC']),2))
        else:
            teams[row['HomeTeam']].append(round((teams[row['HomeTeam']][-1] + row['HC'])/j,2))
            teams[row['AwayTeam']].append(round((teams[row['AwayTeam']][-1] + row['AC'])/j,2))
            
        if (i % 8)== 0:
            j = j + 1
        
    df['avgHC'] = 0
    df['avgAC'] = 0
    
    for i, row in df.iterrows():
    
        df.at[i, 'avgHC'] = teams[row['HomeTeam']].pop(0)
        df.at[i, 'avgAC'] = teams[row['AwayTeam']].pop(0)
  
    return df

In [12]:
s1415 = avg_team_corner(s1415)
s1516 = avg_team_corner(s1516)
s1617 = avg_team_corner(s1617)
s1718 = avg_team_corner(s1718)
s1819 = avg_team_corner(s1819)
s1920 = avg_team_corner(s1920)

In [13]:
#rolling average of corners a team earns in a time window of 3 games
def roll_avg_C(df):
    
    #Dataframe of home team matches
    home_df = df.groupby(['Date','HomeTeam']).sum()['HC'].reset_index()
    home_df.columns = ['Date','Team','C']
    
    #Dataframe of away team matches
    away_df = df.groupby(['Date','AwayTeam']).sum()['AC'].reset_index()
    away_df.columns = ['Date','Team','C']
    
    #Concatenation of the two dataframes and then we sort it by date ascending
    cdf = pd.concat([home_df,away_df],ignore_index=True)
    cdf = cdf.sort_values('Date',ascending=True)

    #calculation of rolling avg for each team
    cdf['ravg_C'] = cdf.groupby('Team')['C'].apply(lambda x: x.rolling(window=3,closed='left').mean())
    
    #Creation of the columns
    df = pd.merge(df,cdf,how='inner',left_on=['Date','HomeTeam'],right_on=['Date','Team'])
    df.rename(columns={'ravg_C':'l3_ravg_HTC'},inplace=True)
    df = df.drop(['Team','C'],axis=1)
    
    df = pd.merge(df,cdf,how='inner',left_on=['Date','AwayTeam'],right_on=['Date','Team'])
    df.rename(columns={'ravg_C':'l3_ravg_ATC'},inplace=True)
    df = df.drop(['Team','C'],axis=1)


    return df

In [14]:
s1415 = roll_avg_C(s1415)
s1516 = roll_avg_C(s1516)
s1617 = roll_avg_C(s1617)
s1718 = roll_avg_C(s1718)
s1819 = roll_avg_C(s1819)
s1920 = roll_avg_C(s1920)

In [15]:
#rolling average of corners a team earns in a time window of 3 games. Only for home or away matches.
def team_C_at_home_and_away(df): 
    
    df['l3avg_TC_ho'] = df.groupby('HomeTeam')['HC'].apply(lambda x: x.rolling(window=3,closed='left').mean()).fillna(0)
    df['l3avg_TC_ao'] = df.groupby('AwayTeam')['AC'].apply(lambda x: x.rolling(window=3,closed='left').mean()).fillna(0)
    
    return df

In [16]:
s1415 = team_C_at_home_and_away(s1415)
s1516 = team_C_at_home_and_away(s1516)
s1617 = team_C_at_home_and_away(s1617)
s1718 = team_C_at_home_and_away(s1718)
s1819 = team_C_at_home_and_away(s1819)
s1920 = team_C_at_home_and_away(s1920)

In [17]:
#number of goals a team scores during the season
def goal_scored(df):
    teams = {key:[0] for key in df['HomeTeam'].unique()}
    
    for i,row in df.iterrows():
        
        teams[row['HomeTeam']].append(teams[row['HomeTeam']][-1] + row['FTHG'])
        teams[row['AwayTeam']].append(teams[row['AwayTeam']][-1] + row['FTAG'])
        
    df['HTGS'] = 0
    df['ATGS'] = 0
    
    for i, row in df.iterrows():
    
        df.at[i, 'HTGS'] = teams[row['HomeTeam']].pop(0)
        df.at[i, 'ATGS'] = teams[row['AwayTeam']].pop(0)
  
    return df

In [18]:
s1415 = goal_scored(s1415)
s1516 = goal_scored(s1516)
s1617 = goal_scored(s1617)
s1718 = goal_scored(s1718)
s1819 = goal_scored(s1819)
s1920 = goal_scored(s1920)

In [19]:
#number of goals a team concedes during the season
def goals_conceded(df):
    teams = {key:[0] for key in df['HomeTeam'].unique()}
    
    for i,row in df.iterrows():
        
        teams[row['HomeTeam']].append(teams[row['HomeTeam']][-1] + row['FTAG'])
        teams[row['AwayTeam']].append(teams[row['AwayTeam']][-1] + row['FTHG'])
        
    df['HTGC'] = 0
    df['ATGC'] = 0
    
    for i, row in df.iterrows():
    
        df.at[i, 'HTGC'] = teams[row['HomeTeam']].pop(0)
        df.at[i, 'ATGC'] = teams[row['AwayTeam']].pop(0)
    
    
    return df

In [20]:
s1415 = goals_conceded(s1415)
s1516 = goals_conceded(s1516)
s1617 = goals_conceded(s1617)
s1718 = goals_conceded(s1718)
s1819 = goals_conceded(s1819)
s1920 = goals_conceded(s1920)

In [21]:
#overall stats for each team
def team_overall(df):
    
    df['HToveral'] = df['HTGS'] - df['HTGC']
    df['AToveral'] = df['ATGS'] - df['ATGC']
    
    return df

In [22]:
s1415 = team_overall(s1415)
s1516 = team_overall(s1516)
s1617 = team_overall(s1617)
s1718 = team_overall(s1718)
s1819 = team_overall(s1819)
s1920 = team_overall(s1920)

In [23]:
#conversion rate of teams
def cr_team(df):
    
    df['HCR'] = np.where(df['HST']!=0, round(df['FTHG']/df['HST'],2),0)
    df['ACR'] = np.where(df['AST']!=0, round(df['FTAG']/df['AST'],2),0)
    
    return df


In [24]:
s1415 = cr_team(s1415)
s1516 = cr_team(s1516)
s1617 = cr_team(s1617)
s1718 = cr_team(s1718)
s1819 = cr_team(s1819)
s1920 = cr_team(s1920)

In [25]:
#rolling average of conversion rate of each team only at home or away games
def team_CR_at_home_and_away(df): 
    
    df['l3avg_Tcr_ho'] = df.groupby('HomeTeam')['HCR'].apply(lambda x: x.rolling(window=3,closed='left').mean()).fillna(0)
    df['l3avg_Tcr_ao'] = df.groupby('AwayTeam')['ACR'].apply(lambda x: x.rolling(window=3,closed='left').mean()).fillna(0)
    
    return df

In [26]:
s1415 = team_CR_at_home_and_away(s1415)
s1516 = team_CR_at_home_and_away(s1516)
s1617 = team_CR_at_home_and_away(s1617)
s1718 = team_CR_at_home_and_away(s1718)
s1819 = team_CR_at_home_and_away(s1819)
s1920 = team_CR_at_home_and_away(s1920)

In [27]:
#average conversion rate of teams from fixture to fixture
def avg_team_cr(df):
    teams = {key:[0] for key in df['HomeTeam'].unique()}
    
    j=0
    for i,row in df.iterrows():
        
        if j==0:
            teams[row['HomeTeam']].append(round((teams[row['HomeTeam']][-1] + row['HCR']),2))
            teams[row['AwayTeam']].append(round((teams[row['AwayTeam']][-1] + row['ACR']),2))
        else:
            teams[row['HomeTeam']].append(round((teams[row['HomeTeam']][-1] + row['HCR'])/j,2))
            teams[row['AwayTeam']].append(round((teams[row['AwayTeam']][-1] + row['ACR'])/j,2))
        
        if (i % 8 == 0):
            j = j + 1
                                            
    df['avgHCR'] = 0
    df['avgACR'] = 0
    
    for i, row in df.iterrows():
    
        df.at[i, 'avgHCR'] = teams[row['HomeTeam']].pop(0)
        df.at[i, 'avgACR'] = teams[row['AwayTeam']].pop(0)
    
    return df

In [28]:
s1415 = avg_team_cr(s1415)
s1516 = avg_team_cr(s1516)
s1617 = avg_team_cr(s1617)
s1718 = avg_team_cr(s1718)
s1819 = avg_team_cr(s1819)
s1920 = avg_team_cr(s1920)

In [29]:
#rolling average of conversion rate in a time window of 3 games.
def roll_avg_CR(df):
    
    #Dataframe of home team matches
    home_df = df.groupby(['Date','HomeTeam']).sum()['HCR'].reset_index()
    home_df.columns = ['Date','Team','CR']
    
    #Dataframe of away team matches
    away_df = df.groupby(['Date','AwayTeam']).sum()['ACR'].reset_index()
    away_df.columns = ['Date','Team','CR']
    
    #Concatenation of the two dataframes and then we sort it by date ascending
    cdf = pd.concat([home_df,away_df],ignore_index=True)
    cdf = cdf.sort_values('Date',ascending=True)

    #calculation of rolling avg for each team
    cdf['ravg_CR'] = cdf.groupby('Team')['CR'].apply(lambda x: x.rolling(window=3,closed='left').mean())
    
    df = pd.merge(df,cdf,how='inner',left_on=['Date','HomeTeam'],right_on=['Date','Team'])
    df.rename(columns={'ravg_CR':'l3_ravg_HTCR'},inplace=True)
    df = df.drop(['Team','CR'],axis=1)
    
    df = pd.merge(df,cdf,how='inner',left_on=['Date','AwayTeam'],right_on=['Date','Team'])
    df.rename(columns={'ravg_CR':'l3_ravg_ATCR'},inplace=True)
    df = df.drop(['Team','CR'],axis=1)


    return df

In [30]:
s1415 = roll_avg_CR(s1415)
s1516 = roll_avg_CR(s1516)
s1617 = roll_avg_CR(s1617)
s1718 = roll_avg_CR(s1718)
s1819 = roll_avg_CR(s1819)
s1920 = roll_avg_CR(s1920)

In [31]:
#rolling avg of stats of each team in a time window of 3 games
def team_stats_at_home_and_away(df): 

    df['avgl3_Tatt_ho'] = df.groupby('HomeTeam')['FTHG'].apply(lambda x: x.rolling(window=3,closed='left').mean()).fillna(0)
    df['avgl3_Tconc_ho'] = df.groupby('HomeTeam')['FTAG'].apply(lambda x: x.rolling(window=3,closed='left').mean()).fillna(0)
    df['avgl3_Tatt_ao'] = df.groupby('AwayTeam')['FTAG'].apply(lambda x: x.rolling(window=3,closed='left').mean()).fillna(0)
    df['avgl3_Tconc_ao'] = df.groupby('AwayTeam')['FTHG'].apply(lambda x: x.rolling(window=3,closed='left').mean()).fillna(0)
    
    return df

In [32]:
s1415 = team_stats_at_home_and_away(s1415)
s1516 = team_stats_at_home_and_away(s1516)
s1617 = team_stats_at_home_and_away(s1617)
s1718 = team_stats_at_home_and_away(s1718)
s1819 = team_stats_at_home_and_away(s1819)
s1920 = team_stats_at_home_and_away(s1920)

In [33]:
#avg xG for each team from fixture to fixture
def avg_team_xG(df):
    teams = {key:[0] for key in df['HomeTeam'].unique()}
    
    j=0
    for i,row in df.iterrows():
        
        if j==0:
            teams[row['HomeTeam']].append(round((teams[row['HomeTeam']][-1] + row['HxG']),2))
            teams[row['AwayTeam']].append(round((teams[row['AwayTeam']][-1] + row['AxG']),2))
        else:
            teams[row['HomeTeam']].append(round((teams[row['HomeTeam']][-1] + row['HxG'])/j,2))
            teams[row['AwayTeam']].append(round((teams[row['AwayTeam']][-1] + row['AxG'])/j,2))
        
        if (i % 8 == 0):
            j = j + 1
                                            
    df['HxG_pm'] = 0
    df['AxG_pm'] = 0
    
    for i, row in df.iterrows():
    
        df.at[i, 'HxG_pm'] = teams[row['HomeTeam']].pop(0)
        df.at[i, 'AxG_pm'] = teams[row['AwayTeam']].pop(0)
    
    #df = df.drop(['HxG','AxG'],axis=1)
    
    return df

In [34]:
s1415 = avg_team_xG(s1415)
s1516 = avg_team_xG(s1516)
s1617 = avg_team_xG(s1617)
s1718 = avg_team_xG(s1718)
s1819 = avg_team_xG(s1819)
s1920 = avg_team_xG(s1920)

In [35]:
#rolling average of xGoals of each team in a time window of 3 games
def roll_avg_xG(df):
    
    #Dataframe of home team matches
    home_df = df.groupby(['Date','HomeTeam']).sum()['HxG'].reset_index()
    home_df.columns = ['Date','Team','xG']
    
    #Dataframe of away team matches
    away_df = df.groupby(['Date','AwayTeam']).sum()['AxG'].reset_index()
    away_df.columns = ['Date','Team','xG']
    
    #Concatenation of the two dataframes and then we sort it by date ascending
    cdf = pd.concat([home_df,away_df],ignore_index=True)
    cdf = cdf.sort_values('Date',ascending=True)

    #calculation of rolling avg for each team
    cdf['ravg_xG'] = cdf.groupby('Team')['xG'].apply(lambda x: x.rolling(window=3,closed='left').mean())
    
    df = pd.merge(df,cdf,how='inner',left_on=['Date','HomeTeam'],right_on=['Date','Team'])
    df.rename(columns={'ravg_xG':'l3_ravg_HTxG'},inplace=True)
    df = df.drop(['Team','xG'],axis=1)
    
    df = pd.merge(df,cdf,how='inner',left_on=['Date','AwayTeam'],right_on=['Date','Team'])
    df.rename(columns={'ravg_xG':'l3_ravg_ATxG'},inplace=True)
    df = df.drop(['Team','xG'],axis=1)


    return df

In [36]:
s1415 = roll_avg_xG(s1415)
s1516 = roll_avg_xG(s1516)
s1617 = roll_avg_xG(s1617)
s1718 = roll_avg_xG(s1718)
s1819 = roll_avg_xG(s1819)
s1920 = roll_avg_xG(s1920)

In [37]:
#rolling avg xG of each team only at home or away turf
#time window of 3 games

def xG_at_home_and_away(df): 
    
    df['l3avg_xG_hpm'] = df.groupby('HomeTeam')['HxG'].apply(lambda x: x.rolling(window=3,closed='left').mean()).fillna(0)
    df['l3avg_xG_apm'] = df.groupby('AwayTeam')['AxG'].apply(lambda x: x.rolling(window=3,closed='left').mean()).fillna(0)
    
    return df

In [38]:
s1415 = xG_at_home_and_away(s1415)
s1516 = xG_at_home_and_away(s1516)
s1617 = xG_at_home_and_away(s1617)
s1718 = xG_at_home_and_away(s1718)
s1819 = xG_at_home_and_away(s1819)
s1920 = xG_at_home_and_away(s1920)

In [39]:
#avg deep of each team from fixture to fixture
def avg_team_deep(df):
    teams = {key:[0] for key in df['HomeTeam'].unique()}
    
    j=0
    for i,row in df.iterrows():
        
        if j==0:
            teams[row['HomeTeam']].append(round((teams[row['HomeTeam']][-1] + row['Hdeep']),2))
            teams[row['AwayTeam']].append(round((teams[row['AwayTeam']][-1] + row['Adeep']),2))
        else:
            teams[row['HomeTeam']].append(round((teams[row['HomeTeam']][-1] + row['Hdeep'])/j,2))
            teams[row['AwayTeam']].append(round((teams[row['AwayTeam']][-1] + row['Adeep'])/j,2))
        
        if (i % 8 == 0):
            j = j + 1
                                            
    df['avgHdeep'] = 0
    df['avgAdeep'] = 0
    
    for i, row in df.iterrows():
    
        df.at[i, 'avgHdeep'] = teams[row['HomeTeam']].pop(0)
        df.at[i, 'avgAdeep'] = teams[row['AwayTeam']].pop(0)
    return df

In [40]:
s1415 = avg_team_deep(s1415)
s1516 = avg_team_deep(s1516)
s1617 = avg_team_deep(s1617)
s1718 = avg_team_deep(s1718)
s1819 = avg_team_deep(s1819)
s1920 = avg_team_deep(s1920)

In [41]:
#rolling average of deep each team makes
#3 games time window

def roll_avg_deep(df):
    
    #Dataframe of home team matches
    home_df = df.groupby(['Date','HomeTeam']).sum()['Hdeep'].reset_index()
    home_df.columns = ['Date','Team','deep']
    
    #Dataframe of away team matches
    away_df = df.groupby(['Date','AwayTeam']).sum()['Adeep'].reset_index()
    away_df.columns = ['Date','Team','deep']
    
    #Concatenation of the two dataframes and then we sort it by date ascending
    cdf = pd.concat([home_df,away_df],ignore_index=True)
    cdf = cdf.sort_values('Date',ascending=True)

    #calculation of rolling avg for each team
    cdf['ravg_deep'] = cdf.groupby('Team')['deep'].apply(lambda x: x.rolling(window=3,closed='left').mean())
    
    df = pd.merge(df,cdf,how='inner',left_on=['Date','HomeTeam'],right_on=['Date','Team'])
    df.rename(columns={'ravg_deep':'l3_ravg_HTdeep'},inplace=True)
    df = df.drop(['Team','deep'],axis=1)
    
    df = pd.merge(df,cdf,how='inner',left_on=['Date','AwayTeam'],right_on=['Date','Team'])
    df.rename(columns={'ravg_deep':'l3_ravg_ATdeep'},inplace=True)
    df = df.drop(['Team','deep'],axis=1)


    return df

In [42]:
s1415 = roll_avg_deep(s1415)
s1516 = roll_avg_deep(s1516)
s1617 = roll_avg_deep(s1617)
s1718 = roll_avg_deep(s1718)
s1819 = roll_avg_deep(s1819)
s1920 = roll_avg_deep(s1920)

In [43]:
#rolling average of deep each team creates
#time window of 3 games
def deep_at_home_and_away(df): 
    #df['deep_hpm'] = df.groupby('HomeTeam')['Hdeep'].apply(lambda x: x.shift().expanding().mean()).fillna(0)
    #df['deep_apm'] = df.groupby('AwayTeam')['Adeep'].apply(lambda x: x.shift().expanding().mean()).fillna(0)
    
    df['avgl5_deep_hpm'] = df.groupby('HomeTeam')['Hdeep'].apply(lambda x: x.rolling(window=3,closed='left').mean()).fillna(0)
    df['avgl5_deep_apm'] = df.groupby('AwayTeam')['Adeep'].apply(lambda x: x.rolling(window=3,closed='left').mean()).fillna(0)
    
    return df

In [44]:
s1415 = deep_at_home_and_away(s1415)
s1516 = deep_at_home_and_away(s1516)
s1617 = deep_at_home_and_away(s1617)
s1718 = deep_at_home_and_away(s1718)
s1819 = deep_at_home_and_away(s1819)
s1920 = deep_at_home_and_away(s1920)

In [45]:
#Average goals a team scores from fixture to fixture
def avg_goal_scored(df):
    teams = {key:[0] for key in df['HomeTeam'].unique()}
    
    j=0
    
    for i,row in df.iterrows():
       
        if j==0:
            teams[row['HomeTeam']].append(round((teams[row['HomeTeam']][-1] + row['FTHG']),2))
            teams[row['AwayTeam']].append(round((teams[row['AwayTeam']][-1] + row['FTAG']),2))
        else:
            teams[row['HomeTeam']].append(round((teams[row['HomeTeam']][-1] + row['FTHG'])/j,2))
            teams[row['AwayTeam']].append(round((teams[row['AwayTeam']][-1] + row['FTAG'])/j,2))
            
        if (i % 8)== 0:
            j = j + 1
        
    df['avgHTGS'] = 0
    df['avgATGS'] = 0
    
    for i, row in df.iterrows():
    
        df.at[i, 'avgHTGS'] = teams[row['HomeTeam']].pop(0)
        df.at[i, 'avgATGS'] = teams[row['AwayTeam']].pop(0)
  
    return df

In [46]:
s1415 = avg_goal_scored(s1415)
s1516 = avg_goal_scored(s1516)
s1617 = avg_goal_scored(s1617)
s1718 = avg_goal_scored(s1718)
s1819 = avg_goal_scored(s1819)
s1920 = avg_goal_scored(s1920)

In [47]:
#Average goals a team concedes from fixture to fixture
def avg_goal_conceded(df):
    teams = {key:[0] for key in df['HomeTeam'].unique()}
    
    j=0
    
    for i,row in df.iterrows():

        if j==0:
            teams[row['HomeTeam']].append(round((teams[row['HomeTeam']][-1] + row['FTAG']),2))
            teams[row['AwayTeam']].append(round((teams[row['AwayTeam']][-1] + row['FTHG']),2))
        else:
            teams[row['HomeTeam']].append(round((teams[row['HomeTeam']][-1] + row['FTAG'])/j,2))
            teams[row['AwayTeam']].append(round((teams[row['AwayTeam']][-1] + row['FTHG'])/j,2))
            
        if (i % 8)== 0:
            j = j + 1

    df['avgHTGC'] = 0
    df['avgATGC'] = 0
    
    for i, row in df.iterrows():
    
        df.at[i, 'avgHTGC'] = teams[row['HomeTeam']].pop(0)
        df.at[i, 'avgATGC'] = teams[row['AwayTeam']].pop(0)
    
  
    return df

In [48]:
s1415 = avg_goal_conceded(s1415)
s1516 = avg_goal_conceded(s1516)
s1617 = avg_goal_conceded(s1617)
s1718 = avg_goal_conceded(s1718)
s1819 = avg_goal_conceded(s1819)
s1920 = avg_goal_conceded(s1920)

In [49]:
#team points
def match_point(home_team, FTR):
    if FTR == 'H':
        if home_team:
            return 3
        else:
            return 0
    elif FTR == 'A':
        if home_team:
            return 0
        else:
            return 3
    else:
        return 1

def team_points(df):
    teams = {key:[0] for key in df['HomeTeam'].unique()}
  
    for i, row in df.iterrows():
    
        teams[row['HomeTeam']].append(teams[row['HomeTeam']][-1] + match_point(home_team=True, FTR=row['FTR']))
        teams[row['AwayTeam']].append(teams[row['AwayTeam']][-1] + match_point(home_team=False, FTR=row['FTR']))
    
    df['HomeTeamPoints'] = 0
    df['AwayTeamPoints'] = 0
    
    for i, row in df.iterrows():
    
        df.at[i, 'HomeTeamPoints'] = teams[row.HomeTeam].pop(0)
        df.at[i, 'AwayTeamPoints'] = teams[row.AwayTeam].pop(0)
  
    return df


In [50]:
s1415 = team_points(s1415)
s1516 = team_points(s1516)
s1617 = team_points(s1617)
s1718 = team_points(s1718)
s1819 = team_points(s1819)
s1920 = team_points(s1920)

In [51]:
#average of team points
def match_point(home_team, FTR):
    if FTR == 'H':
        if home_team:
            return 3
        else:
            return 0
    elif FTR == 'A':
        if home_team:
            return 0
        else:
            return 3
    else:
        return 1

def avg_team_points(df):
    teams = {key:[0] for key in df['HomeTeam'].unique()}
  
    j=0
    for i, row in df.iterrows():
        
   
        
        if j==0:
            teams[row['HomeTeam']].append(teams[row['HomeTeam']][-1] + match_point(home_team=True, FTR=row['FTR']))
            teams[row['AwayTeam']].append(teams[row['AwayTeam']][-1] + match_point(home_team=False, FTR=row['FTR']))
        else:
            teams[row['HomeTeam']].append(round((teams[row['HomeTeam']][-1] + match_point(home_team=True, FTR=row['FTR']))/j,2))
            teams[row['AwayTeam']].append(round((teams[row['AwayTeam']][-1] + match_point(home_team=False, FTR=row['FTR']))/j,2))
    
        if (i % 8) == 0:
            j = j + 1
    
    df['avgHTP'] = 0
    df['avgATP'] = 0
    
    for i, row in df.iterrows():
    
        df.at[i, 'avgHTP'] = teams[row.HomeTeam].pop(0)
        df.at[i, 'avgATP'] = teams[row.AwayTeam].pop(0)
  
    return df

In [52]:
s1415 = avg_team_points(s1415)
s1516 = avg_team_points(s1516)
s1617 = avg_team_points(s1617)
s1718 = avg_team_points(s1718)
s1819 = avg_team_points(s1819)
s1920 = avg_team_points(s1920)

In [53]:
#last 5 games form 
from copy import deepcopy

def match_FTR(home_team, FTR):
    if FTR == 'H':
        if home_team:
            return 'W'
        else:
            return 'L'
    elif FTR == 'A':
        if home_team:
            return 'L'
        else:
            return'W'
    else:
        return 'D'

def last_5_games_form(df):
    teams = {key:['-'] for key in df.HomeTeam.unique()} #- since in the begining of the season there is no game history
  
    for i, row in df.iterrows():
        teams[row['HomeTeam']].append(match_FTR(home_team = True, FTR = row['FTR']))
        teams[row['AwayTeam']].append(match_FTR(home_team = False, FTR = row['FTR']))
    
  
    teams2 = deepcopy(teams)
    teams3 = deepcopy(teams)
    teams4 = deepcopy(teams)
    teams5 = deepcopy(teams)   
    
    for key in teams:
        teams2[key].insert(0,'-')
    
        teams3[key].insert(0,'-')
        teams3[key].insert(0,'-')
    
        teams4[key].insert(0,'-')
        teams4[key].insert(0,'-')
        teams4[key].insert(0,'-')
    
        teams5[key].insert(0,'-')
        teams5[key].insert(0,'-')
        teams5[key].insert(0,'-')
        teams5[key].insert(0,'-')

  # initialize last 5 games to '-'
    for i in range(1, 6):
        df['HM' + str(i)] = '-'
        df['AM' + str(i)] = '-'
  
    for i, row in df.iterrows():
    
        df.at[i, 'HM5'] = teams5[row['HomeTeam']].pop(0)
        df.at[i, 'HM4'] = teams4[row['HomeTeam']].pop(0)
        df.at[i, 'HM3'] = teams3[row['HomeTeam']].pop(0)
        df.at[i, 'HM2'] = teams2[row['HomeTeam']].pop(0)
        df.at[i, 'HM1'] = teams[row['HomeTeam']].pop(0)
    
        df.at[i, 'AM5'] = teams5[row['AwayTeam']].pop(0)
        df.at[i, 'AM4'] = teams4[row['AwayTeam']].pop(0)
        df.at[i, 'AM3'] = teams3[row['AwayTeam']].pop(0)
        df.at[i, 'AM2'] = teams2[row['AwayTeam']].pop(0)
        df.at[i, 'AM1'] = teams[row['AwayTeam']].pop(0)
  
    return df

In [54]:
s1415 = last_5_games_form(s1415)
s1516 = last_5_games_form(s1516)
s1617 = last_5_games_form(s1617)
s1718 = last_5_games_form(s1718)
s1819 = last_5_games_form(s1819)
s1920 = last_5_games_form(s1920)

In [55]:
df = pd.concat([s1415, s1516, s1617, s1718, s1819, s1920]).reset_index(drop=True)

In [56]:
#difference of points between the home and away team
def diff_points(df):
    
    df['diff_points'] = df['HomeTeamPoints'] - df['AwayTeamPoints']
    
    return df

In [57]:
df = diff_points(df)

In [58]:
df

,Date,HomeTeam,AwayTeam,FTHG,FTAG,HTHG,HTAG,FTR,HS,AS,...,AM1,HM2,AM2,HM3,AM3,HM4,AM4,HM5,AM5,diff_points
0,2014-08-22,Bayern Munich,Wolfsburg,2,1,1,0,H,17,9,...,-,-,-,-,-,-,-,-,-,0
1,2014-08-23,Dortmund,Leverkusen,0,2,0,1,A,11,11,...,-,-,-,-,-,-,-,-,-,0
2,2014-08-23,Ein Frankfurt,Freiburg,1,0,1,0,H,10,13,...,-,-,-,-,-,-,-,-,-,0
3,2014-08-23,FC Koln,Hamburg,0,0,0,0,D,10,19,...,-,-,-,-,-,-,-,-,-,0
4,2014-08-23,Hannover,Schalke 04,2,1,0,0,H,8,12,...,-,-,-,-,-,-,-,-,-,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1831,2020-06-27,Leverkusen,Mainz,1,0,1,0,H,13,6,...,W,W,W,D,L,L,W,W,L,23
1832,2020-06-27,M'gladbach,Hertha,2,1,1,0,H,22,4,...,W,W,L,L,L,L,L,W,W,21
1833,2020-06-27,Union Berlin,Fortuna Dusseldorf,3,0,1,0,H,11,25,...,D,W,D,W,L,D,D,L,L,8
1834,2020-06-27,Werder Bremen,FC Koln,6,1,3,0,H,18,6,...,D,L,L,W,L,L,D,L,L,-8


In [59]:
#points a team gathered after 5 games
for i in range(1,6):
    df["HMP" + str(i)] = df["HM" + str(i)].map({'W':3, 'D':1, 'L':0, '-':0})
    df['AMP' + str(i)] = df["AM" + str(i)].map({'L':0, 'D':1, 'W':3, '-':0})
    
df['Form_HTP'] = df['HMP1'] + df['HMP2'] + df['HMP3'] + df['HMP4'] + df['HMP5']
df['Form_ATP'] = df['AMP1'] + df['AMP2'] + df['AMP3'] + df['AMP4'] + df['AMP5']

df = df.drop(['HMP1','HMP2','HMP3','HMP4','HMP5','AMP1','AMP2','AMP3','AMP4','AMP5'],axis=1)

In [60]:
df['Fomr_Points_diff'] = df['Form_HTP'] - df['Form_ATP']

In [61]:
df = df.drop(['HS','AS','HST','AST','HC','AC','HF','AF','HxG','AxG',
              'Hdeep','Adeep',],axis=1)


In [62]:
df['HomeTeamForm'] = df['HM1'] + df['HM2'] + df['HM3'] + df['HM4'] + df['HM5']
df['AwayTeamForm'] = df['AM1'] + df['AM2'] + df['AM3'] + df['AM4'] + df['AM5']


In [63]:
df = df.drop(['HM1', 'HM2', 'HM3', 'HM4', 'HM5', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5'], axis=1)

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1836 entries, 0 to 1835
Data columns (total 74 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              1836 non-null   object 
 1   HomeTeam          1836 non-null   object 
 2   AwayTeam          1836 non-null   object 
 3   FTHG              1836 non-null   int64  
 4   FTAG              1836 non-null   int64  
 5   HTHG              1836 non-null   int64  
 6   HTAG              1836 non-null   int64  
 7   FTR               1836 non-null   object 
 8   league            1836 non-null   object 
 9   Hppda_coef        1836 non-null   float64
 10  Appda_coef        1836 non-null   float64
 11  AVGH              1836 non-null   float64
 12  AVGD              1836 non-null   float64
 13  AVGA              1836 non-null   float64
 14  season            1836 non-null   object 
 15  ranking_HT_ls     1836 non-null   float64
 16  ranking_AT_ls     1836 non-null   float64


In [65]:
df.to_csv("bundesliga.csv")